In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

Setting environment for Gordon


No vtk


In [2]:
stack_fixed = 'MD593'

In [3]:
sec2fn = metadata_cache['sections_to_filenames'][stack_fixed]

In [4]:
train_sample_scheme = 1
global_transform_scheme = 1

local_transform_scheme = 2

reg_weights = np.array([0.,0.,0.])
    
# stack_moving = 'atlas_on_MD589'
stack_moving = 'atlasV2'

In [5]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

structures_sided = sum([[n] if n in singular_structures 
                        else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

structures_sided_plus_surround = sum([[s, s+'_surround'] for s in structures_sided], [])

In [6]:
label_to_name_fixed = {i+1: name for i, name in enumerate(sorted(structures))}
name_to_label_fixed = {n:l for l, n in label_to_name_fixed.iteritems()}

In [7]:
# if local_transform_scheme == 1 or local_transform_scheme == 2:
    
# label_to_name_moving = {i+1: name for i, name in enumerate(structures_sided)}
# name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

# elif local_transform_scheme == 3 or local_transform_scheme == 4:

label_to_name_moving = {i+1: name for i, name in enumerate(structures_sided_plus_surround)}
name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

In [8]:
# Load fixed volumes

volume_fixed = {name_to_label_fixed[name]: DataManager.load_score_volume(stack=stack_fixed, label=name, downscale=32, train_sample_scheme=train_sample_scheme)
               for name in structures}

print volume_fixed.values()[0].shape
print volume_fixed.values()[0].dtype

vol_fixed_ydim, vol_fixed_xdim, vol_fixed_zdim = volume_fixed.values()[0].shape

# vol_fixed_xmin, vol_fixed_ymin, vol_fixed_zmin = (0,0,0)
# vol_fixed_ymax, vol_fixed_xmax, vol_fixed_zmax = np.array(volume_fixed.values()[0].shape) - 1
# vol_fixed_xdim = vol_fixed_xmax + 1 - vol_fixed_xmin
# vol_fixed_ydim = vol_fixed_ymax + 1 - vol_fixed_ymin
# vol_fixed_zdim = vol_fixed_zmax + 1 - vol_fixed_zmin

(383, 534, 374)
float16


In [9]:
# Load moving volumes

# if local_transform_scheme == 3 or local_transform_scheme == 4:

volume_moving = {name_to_label_moving[name_s]: DataManager.load_transformed_volume(stack_m=stack_moving,
                                                                                   type_m='score',
                                                                                   stack_f=stack_fixed,
                                                                                   type_f='score',
                                                                                   downscale=32,
                                                                                   train_sample_scheme_f=train_sample_scheme,
                                                                                   global_transform_scheme=global_transform_scheme,
                                                                                   label=name_s,
                                                                                  local_transform_scheme=2,
                                                                                  transitive='over')
                 for name_s in structures_sided_plus_surround}

# else:

# volume_moving = {name_to_label_moving[name_s]: \
#                  DataManager.load_transformed_volume(stack_m=stack_moving,
#                                                    type_m='score',
#                                                    stack_f=stack_fixed,
#                                                    type_f='score',
#                                                    downscale=32,
#                                                    train_sample_scheme_f=train_sample_scheme,
#                                                    global_transform_scheme=global_transform_scheme,
#                                                    local_transform_scheme=2,
#                                                    label=name_s,
#                                                   transitive='over')
#                  for name_s in structures_sided}    
    
print volume_moving.values()[0].shape
print volume_moving.values()[0].dtype

# vol_moving_xmin, vol_moving_ymin, vol_moving_zmin = (0,0,0)
# vol_moving_ymax, vol_moving_xmax, vol_moving_zmax = np.array(volume_moving.values()[0].shape) - 1

(383, 534, 374)
float16


In [10]:
# Load moving volumes

# if local_transform_scheme == 3 or local_transform_scheme == 4:

#     volume_moving = {name_to_label_moving[name_s]: DataManager.load_transformed_volume(stack_m=stack_moving,
#                                                                                        type_m='score',
#                                                                                        stack_f=stack_fixed,
#                                                                                        type_f='score',
#                                                                                        downscale=32,
#                                                                                        train_sample_scheme_f=train_sample_scheme,
#                                                                                        global_transform_scheme=global_transform_scheme,
#                                                                                        label=name_s)
#                      for name_s in structures_sided_plus_surround}

# else:

volume_moving_gtf = {name_to_label_moving[name_s]: \
                 DataManager.load_transformed_volume(stack_m=stack_moving,
                                                   type_m='score',
                                                   stack_f=stack_fixed,
                                                   type_f='score',
                                                   downscale=32,
                                                   train_sample_scheme_f=train_sample_scheme,
                                                   global_transform_scheme=global_transform_scheme,
                                                   label=name_s,
                                                  transitive='to')
#                      for name_s in structures_sided_plus_surround}
                 for name_s in structures_sided}    
                    

print volume_moving_gtf.values()[0].shape
print volume_moving_gtf.values()[0].dtype

# vol_moving_xmin, vol_moving_ymin, vol_moving_zmin = (0,0,0)
# vol_moving_ymax, vol_moving_xmax, vol_moving_zmax = np.array(volume_moving.values()[0].shape) - 1

(383, 534, 374)
float16


In [11]:
labelIndexMap_m2f = {}
for label_m, name_m in label_to_name_moving.iteritems():
    labelIndexMap_m2f[label_m] = name_to_label_fixed[convert_to_original_name(name_m)]
    
label_weights_m = {}
for label_m, name_m in label_to_name_moving.iteritems():
    if 'surround' in name_m:
#         if local_transform_scheme == 3 or local_transform_scheme == 4:
        label_weights_m[label_m] = -1
#         else:
#             label_weights_m[label_m] = 0
    else:
        label_weights_m[label_m] = 1

In [26]:
# volume_fixed_sliced = {l: v[..., [160]] for l, v in volume_fixed.iteritems()}
# volume_moving_sliced = {l: v[..., [160]] for l, v in volume_moving.iteritems()}

# Transform

In [62]:
#     transformed_structure_slices = defaultdict(dict)

In [ ]:
for name_s in structures_sided:
# for name_s in ['7N_L']:

    SLICE_TF_VIZ_DIR = create_if_not_exists('/home/yuncong/csd395/CSHL_sliceTfViz/%(stack)s/%(name_s)s' % \
                                dict(stack=stack_fixed, name_s=name_s))

    # For getting correct contour location
    xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = \
    DataManager.load_volume_bbox(stack=stack_fixed, type='score', label=convert_to_original_name(name_s), downscale=32)
    print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

#     transformed_structure_slices = {}
    
    sys.stderr.write('%s\n' % name_s)

    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(volume_moving[name_to_label_moving[name_s]])
    print 'Structure bnounding box:', xmin, xmax, ymin, ymax, zmin, zmax

    sec_min = DataManager.convert_z_to_section(stack=stack_fixed, z=zmin, downsample=32)
    sec_max = DataManager.convert_z_to_section(stack=stack_fixed, z=zmax, downsample=32)
    print sec_min, sec_max
    
# first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]
    bar = show_progress_bar(sec_min, sec_max)

    for sec in range(sec_min, sec_max+1):
    # for sec in range(143, 149):
#     for sec in [176]:
    
        sys.stderr.write('%s\n' % sec)

        bar.value = sec

        zl, zh = DataManager.convert_section_to_z(stack=stack_fixed, sec=sec, downsample=32)
        z = int(np.round((zl + zh)/2))

        # align with z=z
    
#         if local_transform_scheme == 1 or local_transform_scheme == 2:
    
#             aligner = Aligner4(volume_fixed, {name_to_label_moving[name_s]: 
#                                               volume_moving[name_to_label_moving[name_s]]}, \
#                                labelIndexMap_m2f={name_to_label_moving[name_s]:
#                                                   name_to_label_fixed[convert_name_to_unsided(name_s)]},
#                               zrange=(z,z))

#         elif local_transform_scheme == 3 or local_transform_scheme == 4:
        
        try:
            aligner = Aligner4(volume_fixed, {name_to_label_moving[name_s]: volume_moving[name_to_label_moving[name_s]],
                                             name_to_label_moving[name_s+'_surround']: volume_moving[name_to_label_moving[name_s+'_surround']]}, \
                            labelIndexMap_m2f={name_to_label_moving[name_s]: name_to_label_fixed[convert_name_to_unsided(name_s)],
                                              name_to_label_moving[name_s+'_surround']: name_to_label_fixed[convert_to_original_name(name_s+'_surround')]},
                              zrange=(z,z))
        except Exception as e:
            sys.stderr.write('%s\n' % e)
            continue
        
#         aligner = Aligner4(volume_fixed, {name_to_label_moving[name_s]: 
#                                           volume_moving[name_to_label_moving[name_s]]}, \
#                            labelIndexMap_m2f={name_to_label_moving[name_s]:
#                                               name_to_label_fixed[convert_name_to_unsided(name_s)]},
#                           zrange=(z,z))

        # aligner.set_centroid(centroid_m='volume_centroid', centroid_f='volume_centroid')
        aligner.set_centroid(centroid_m='structure_centroid', centroid_f='centroid_m', 
                             indices_m=[name_to_label_moving[name_s]])

        gradient_filepath_map_f = {ind_f: VOLUME_ROOTDIR + '/%(stack)s/score_volume_gradients/%(stack)s_down32_scoreVolume_%(label)s_trainSampleScheme_%(scheme)d_%%(suffix)s.bp' % \
                               {'stack': stack_fixed, 'label': label_to_name_fixed[ind_f], 'scheme':train_sample_scheme}
                               for ind_m, ind_f in labelIndexMap_m2f.iteritems()}

        aligner.load_gradient(gradient_filepath_map_f=gradient_filepath_map_f, indices_f=None)

        t = time.time()
        T, scores = aligner.optimize(type='affine', max_iter_num=1000, history_len=100, terminate_thresh=1e-5,
                                     indices_m=None,
    #                                 grid_search_iteration_number=20,
    #                                  grid_search_sample_number=1000,
    #                                 std_tx=10, std_ty=10, std_tz=0, std_theta_xy=np.deg2rad(10),
                                     grad_computation_sample_number=1e5,
                                     full_lr=[.1,.1,0,1,.1,.1,0,1,0,0,0,0],
                                    label_weights=label_weights_m,
                                    reg_weights=reg_weights,
                                    epsilon=1e-8)

    #     T, scores = aligner.optimize(type='rigid', max_iter_num=1000, history_len=100, terminate_thresh=1e-6,
    #                                      indices_m=None,
    #                                     grid_search_iteration_number=20,
    #                                      grid_search_sample_number=1000,
    #                                     std_tx=5, std_ty=5, std_tz=0, std_theta_xy=np.deg2rad(5),
    #                                      grad_computation_sample_number=1e5,
    #                                      full_lr=[20,20,0,0,0,.1],
    #                                     label_weights=label_weights_m,
    #                                     reg_weights=reg_weights,
    #                                     epsilon=1e-8)

        sys.stderr.write('%.2f seconds\n' % (time.time() - t))

        print T.reshape((3,4))
#         plt.plot(scores);
        print max(scores), scores[-1]
    
        local_params, centroid_m, centroid_f, xdim_f, ydim_f = \
        T.reshape((3,4))[:2,[0,1,3]].flatten(), aligner.centroid_m, aligner.centroid_f, \
        aligner.xdim_f, aligner.ydim_f

        transformed_structure_slice = transform_slice(volume_moving[name_to_label_moving[name_s]][..., z], 
                        T=local_params, centroid_m=aligner.centroid_m[:2], centroid_f=aligner.centroid_f[:2], 
                        xdim_f=xdim_f, ydim_f=ydim_f)
        
        
#     ######## Visualize ########
    
        zl, zh = DataManager.convert_section_to_z(stack=stack_fixed, sec=sec, downsample=32)
        z = int(np.round((zl + zh)/2))

        try:
            # img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
            img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='lossless', version='compressed')
            img = imread(img_fn)[::4, ::4]
        except Exception as e:
            sys.stderr.write('%s\n' % e)
            continue


        viz = img.copy()


        gtf_slice = volume_moving_gtf[name_to_label_moving[name_s]][..., z]

        for level in levels:
            cnts = find_contours(gtf_slice, level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
        #         cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)*8], True, (0,0,255), 1)


        ltf_slice = volume_moving[name_to_label_moving[name_s]][..., z]

        for level in levels:
            cnts = find_contours(ltf_slice, level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
        #         cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)*8], True, (0,255,0), 1)


        # Find moving volume annotation contours
        #     for name_s, vol in vols_m_aligned_to_f.iteritems():
        for level in levels:
            cnts = find_contours(transformed_structure_slice, level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
        #         cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)*8], True, (255,0,0), 1)

    #     display_image(viz)

    #     plt.figure(figsize=(20,20));
    #     plt.imshow(viz);
    #     plt.title('z=%d, sec=%d' % (z, sec));
    #     plt.show()

        viz_fn = os.path.join(SLICE_TF_VIZ_DIR, '%(fn)s_%(name_s)s_sliceTfViz.jpg' % \
                              {'fn': sec2fn[sec], 'name_s': name_s})
        imsave(viz_fn, viz)

# Visualize

In [12]:
# Set colors for different contour level
levels = [0.1, 0.25, 0.5, 0.75, .99]
level_colors = {level: (int(level*255),0,0) for level in levels}

In [61]:
# For getting correct contour location
xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = \
DataManager.load_volume_bbox(stack=stack_fixed, type='score', label=convert_to_original_name(name_s), downscale=32)
print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

# Generate overlay visualization

# Generate atlas overlay image for every section
# first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]
# bar = show_progress_bar(first_sec, last_sec)

# for sec in range(first_sec, last_sec+1):
# for sec in [146]:
#     zl, zh = DataManager.convert_section_to_z(stack=stack_fixed, sec=sec, downsample=32)
#     z = int(np.round((zl + zh)/2))

for sec, transformed_structure_slice_allLabels in transformed_structure_slices.iteritems():
    
    for name_s, transformed_structure_slice in transformed_structure_slice_allLabels.iteritems():
        
        SLICE_TF_VIZ_DIR = create_if_not_exists('/home/yuncong/csd395/CSHL_sliceTfViz/%(stack)s/%(name_s)s' % \
                                        dict(stack=stack_fixed, name_s=name_s))
        
        zl, zh = DataManager.convert_section_to_z(stack=stack_fixed, sec=sec, downsample=32)
        z = int(np.round((zl + zh)/2))

    # if metadata_cache['sections_to_filenames'][stack_fixed][sec] in ['Placeholder', 'Rescan', 'Nonexisting']:
    #     continue

    #     sec = DataManager.convert_z_to_section(stack=stack_fixed, z=z, downsample=32)

    #     bar.value = sec

        # if z < roi_zmin or z > roi_zmax:
        #     continue


        try:
            # img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
            img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='lossless', version='compressed')
            img = imread(img_fn)[::4, ::4]
        except Exception as e:
            sys.stderr.write('%s\n' % e)
            continue


        viz = img.copy()

        
        # gtf contour color: blue
        gtf_slice = volume_moving_gtf[name_to_label_moving[name_s]][..., z]

        for level in levels:
            cnts = find_contours(gtf_slice, level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
        #         cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)*8], True, (0,0,255), 1)


        # ltf contour color: green
        ltf_slice = volume_moving[name_to_label_moving[name_s]][..., z]

        for level in levels:
            cnts = find_contours(ltf_slice, level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
        #         cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)*8], True, (0,255,0), 1)

        # 2D-affine transformed contour color: red
        # Find moving volume annotation contours
        #     for name_s, vol in vols_m_aligned_to_f.iteritems():
        for level in levels:
            cnts = find_contours(transformed_structure_slice, level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
        #         cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)*8], True, (255,0,0), 1)

    #     display_image(viz)

    #     plt.figure(figsize=(20,20));
    #     plt.imshow(viz);
    #     plt.title('z=%d, sec=%d' % (z, sec));
    #     plt.show()

        viz_fn = os.path.join(SLICE_TF_VIZ_DIR, '%(fn)s_%(name_s)s_sliceTfViz.jpg' % \
                              {'fn': sec2fn[sec], 'name_s': name_s})
        imsave(viz_fn, viz)

0 533 0 382 111 484


Section is invalid: Placeholder.
